# _Clean & Store in BigQuery_

In [8]:
import utils

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%%time
#utils.download_json('2020-05-20')

Blob dailies/2020-05-20/2020-05-20_clean-dataset.json downloaded to /home/jupyter/covid_disinfo_detect/experiments/playground_data/2020-05-20_clean-dataset.json.
CPU times: user 24.5 s, sys: 8.12 s, total: 32.6 s
Wall time: 54.9 s


In [9]:
%%time
df = utils.clean_panacea_data(utils.load_data('2020-05-20_clean-dataset.json'))

CPU times: user 4min 23s, sys: 16.8 s, total: 4min 40s
Wall time: 4min 39s


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009492 entries, 0 to 1009491
Data columns (total 28 columns):
 #   Column                        Non-Null Count    Dtype              
---  ------                        --------------    -----              
 0   created_at                    1009492 non-null  datetime64[ns, UTC]
 1   favorite_count                1009492 non-null  int64              
 2   full_text                     1009492 non-null  object             
 3   id_str                        1009492 non-null  object             
 4   in_reply_to_screen_name       1009492 non-null  object             
 5   in_reply_to_status_id_str     1009492 non-null  object             
 6   is_quote_status               1009492 non-null  bool               
 7   lang                          1009492 non-null  object             
 8   retweet_count                 1009492 non-null  int64              
 9   quoted_status_id_str          1009492 non-null  object             
 10  hashta

In [12]:
df.isna().sum()

created_at                      0
favorite_count                  0
full_text                       0
id_str                          0
in_reply_to_screen_name         0
in_reply_to_status_id_str       0
is_quote_status                 0
lang                            0
retweet_count                   0
quoted_status_id_str            0
hashtags                        0
symbols                         0
urls                            0
user_mentions                   0
tweet_source                    0
user_created_at                 0
user_description                0
user_favourites_count           0
user_followers_count            0
user_friends_count              0
user_id_str                     0
user_location                   0
user_name                       0
user_profile_image_url_https    0
user_screen_name                0
user_statuses_count             0
user_verified                   0
quoted_status_url               0
dtype: int64